# <b>Quantization</b>

<b>Required libaries:</b>

In [1]:
!pip install ultralytics
!pip install onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00


<b>Necessary imports:</b>

In [2]:
import os
import numpy as np
import cv2
import torch
from google.colab import drive
from onnxruntime.quantization import CalibrationDataReader, CalibrationMethod, QuantType, QuantFormat, quantize_static
from ultralytics import YOLO
from pycocotools.coco import COCO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# establish connection to google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load and preprocess the model

In [4]:
# load the model
model=YOLO('yolov8x.pt')

# export the model to onnx format
model.export(format='onnx')

100%|██████████| 131M/131M [00:00<00:00, 213MB/s]


Ultralytics 8.3.19 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs

PyTorch: starting from 'yolov8x.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (130.5 MB)
requirements: Ultralytics requirement ['onnxslim'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 4.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 4.6s, installed 1 package: ['onnxslim']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.35...
ONNX: export success ✅ 18.8s, saved as 'yolov8x.onnx' (260.4 MB)

Export complete (30.8s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8x.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8x.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netr

'yolov8x.onnx'

In [5]:
# preprocess the model
!python -m onnxruntime.quantization.preprocess --input yolov8x.onnx --output yolov8x_preprocessed.onnx

### Define class and functions

In [6]:
class ImageCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_paths, target_size=(640, 640)):
        """
        Initialize the calibration data reader.

        :param image_paths: List of paths to the calibration images.
        :param target_size: The target size to resize the images (width, height).
        """
        self.image_paths = image_paths
        self.target_size = target_size
        self.index = 0
        self.input_name = "images"

    def preprocess(self, image_path):
        """
        Preprocess the image: load, resize, normalize, and reformat for the model.

        :param image_path: The path to the image to preprocess.
        :return: Preprocessed image ready for input to the model.
        """
        image = cv2.imread(image_path)

        if image is None:
            raise ValueError(f"Image at path '{image_path}' could not be loaded.")

        # resize the image to the target size
        image = cv2.resize(image, self.target_size)

        # normalise image to [0, 1] range
        image_data = image.astype(np.float32) / 255.0

        # convert image from (H, W, C) to (C, H, W)
        image_data = np.transpose(image_data, (2, 0, 1))

        # add a batch dimension (1, C, H, W)
        image_data = np.expand_dims(image_data, axis=0)

        return image_data

    def get_next(self):
        """
        Get the next batch of data for calibration.

        :return: A dictionary with the model's input tensor name as key and input data as value.
        """
        if self.index >= len(self.image_paths):
            return None

        # preprocess the next image
        image_path = self.image_paths[self.index]
        input_data = self.preprocess(image_path)

        # move to the next image
        self.index += 1

        return {self.input_name: input_data}


In [7]:
def get_calibration_images(annotation_file, image_dir, num_images=100):
    """
    Load the COCO dataset and return paths to the first `num_images` images.

    :param annotation_file: Path to the COCO annotations JSON file.
    :param image_dir: Directory where the COCO images are stored.
    :param num_images: Number of images to select for calibration.
    :return: List of image file paths.
    """
    coco = COCO(annotation_file)
    image_ids = sorted(coco.getImgIds())
    selected_image_ids = image_ids[:num_images]

    calibration_image_paths = []
    for img_id in selected_image_ids:
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(image_dir, img_info['file_name'])
        calibration_image_paths.append(img_path)

    return calibration_image_paths


### Create the ImageCalibrationDataReader

In [8]:
# specify the directories
annotation_file = 'drive/MyDrive/PA2/datasets/coco/annotations/instances_val2017.json'
image_dir = 'drive/MyDrive/PA2/datasets/coco/val2017/'

# get the calibration images
calibration_image_paths = get_calibration_images(annotation_file, image_dir, num_images=100)

# create an instance of the ImageCalibrationDataReader
calibration_data_reader_1 = ImageCalibrationDataReader(calibration_image_paths)


loading annotations into memory...
Done (t=2.66s)
creating index...
index created!


### Quantize the model from fp32 to int8

In [9]:
quantize_static('yolov8x_preprocessed.onnx',                # input fp32 model
                'yolov8x_quantized.onnx',                   # output int8 model
                calibration_data_reader_1,                  # calibration data reader
                quant_format=QuantFormat.QDQ,               # quant format
                reduce_range=True,                          # reduce quant range
                per_channel=True,                           # channelwise quant
                activation_type=QuantType.QUInt8,           # uint8 for activations
                weight_type=QuantType.QInt8,                # int8 for weights
                nodes_to_exclude=[                          # nodes to exclude from quant
                    '/model.22/Concat_3',
                    '/model.22/Split',
                    '/model.22/Sigmoid',
                    '/model.22/dfl/Reshape',
                    '/model.22/dfl/Transpose',
                    '/model.22/dfl/Softmax',
                    '/model.22/dfl/conv/Conv',
                    '/model.22/dfl/Reshape_1',
                    '/model.22/Slice_1',
                    '/model.22/Slice',
                    '/model.22/Add_1',
                    '/model.22/Sub',
                    '/model.22/Div_1',
                    '/model.22/Concat_4',
                    '/model.22/Mul_2',
                    '/model.22/Concat_5'
                ],
                calibrate_method=CalibrationMethod.MinMax)   # MinMax calibration method
